In [87]:
import json 
import pandas as pd
import matplotlib.pyplot as plt
import re

In [88]:
#데이터 불러오기
with open("../data/wikipedia_documents.json") as json_file:
    json_data = json.load(json_file)
    
df_wiki = pd.DataFrame(json_data)
df_wiki = df_wiki.transpose()
print(len(df_wiki))

60613


In [89]:
#겹치는 데이터 개수 
all_duplicates = df_wiki[df_wiki.duplicated(subset=['text'], keep=False)]

In [90]:
# 중복 데이터 삭제
df = df_wiki.drop_duplicates(subset=['text'], keep='first').reset_index(drop=True)

#df_duplicates = df[df.duplicated(subset=['text'], keep=False)]

# 중복 제거 후 데이터 개수 : 56737

In [91]:
df_drop = df.drop(['domain','author','html','url','document_id','corpus_source'],axis=1)
df_drop['text_len'] = df_drop['text'].apply(len)

In [92]:
# \n과 \\n을 제거 .
df_drop['text'] = df_drop['text'].str.replace(r'\\n|\n', '', regex=True)

In [93]:
# 한국어 + 숫자 + 특수기호
def korean_ratio(text):
    korean_number_special_count = len(re.findall(r"[가-힣0-9,\.\/'\"#\(\)\!\^\*\@\+\`\_\-\~\$\%\&\=\:\;\?\>\< ]", text))
    total_count = len(text)
    return korean_number_special_count / total_count

df_drop['korean_ratio'] = df_drop['text'].apply(korean_ratio)
average_korean_ratio = df_drop['korean_ratio'].mean()
print('average_korean_ratio : ',average_korean_ratio)

average_korean_ratio :  0.9347557296843726


In [94]:
df_drop['korean_ratio'].describe()

count    56737.000000
mean         0.934756
std          0.097744
min          0.000955
25%          0.917431
50%          0.968992
75%          0.992748
max          1.000000
Name: korean_ratio, dtype: float64

In [95]:
# only 한국어

# def korean_ratio(text):
#     korean_number_special_count = len(re.findall(r"[가-힣]", text))
#     total_count = len(text)
#     return korean_number_special_count / total_count

# df_drop['korean_ratio'] = df_drop['text'].apply(korean_ratio)
# average_korean_ratio = df_drop['korean_ratio'].mean()
# print('average_korean_ratio : ',average_korean_ratio)

#print(df_drop['korean_ratio'].describe())

In [96]:
#특수문자 기호

# def korean_ratio(text):
#     pattern2 = r"[\.\/'\n\"#\(\)\!\^\*\@\+\`\_\-\~\$\%\&\=\:\;\?\>\< ]"
#     korean_number_special_count = len(re.findall(pattern2, text))
#     total_count = len(text)
#     return korean_number_special_count / total_count

# df_drop['korean_ratio'] = df_drop['text'].apply(korean_ratio)

# average_korean_ratio = df_drop['korean_ratio'].mean()
# print('average_korean_ratio : ',average_korean_ratio)

# print(df_drop['korean_ratio'].describe())

In [97]:
korean_ratio_prepro = df_drop[df_drop['korean_ratio'] > 0.40]

korean_ratio_prepro.to_csv('korean_ratio_0.40_up.csv', index=False, encoding='utf-8')

In [98]:
# #train 데이터셋 정답답과 비교 

# from datasets import load_from_disk
# dataset = load_from_disk("../data/train_dataset/")

# # 'train' 데이터셋에서 context를 시리즈로 가져오기
# contexts = dataset['train']['context']

# # korean_ratio가 0.5보다 작은 텍스트를 가져오기
# korean_ratio_texts = df_drop[df_drop['korean_ratio'] < 0.5]['text']
# #korean_ratio_texts = df_drop['text']
# # 두 시리즈 간의 교집합 찾기
# matching_texts = set(contexts).intersection(set(korean_ratio_texts))

# # 결과 출력
# if len(matching_texts) > 0:
#     print(f"일치하는 텍스트가 {len(matching_texts)}개 있습니다.")
# else:
#     print("일치하는 텍스트가 없습니다.")
